# Depedencies

In [43]:
import pandas as pd
import numpy as np
import time
import collections
import plotly.graph_objects as go
import plotly.express as px
import math
import seaborn as sn

# Import the Data from the excel

In [44]:
# Read the printify products excel file into pandas dataframe
orders = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Orders')
orders.head()

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2013244.178,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1358429.695,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,939390.193,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1856107.400,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1843904.400,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56


In [45]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13506 entries, 0 to 13505
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   MERCHANT_ID             13506 non-null  int64         
 1   ORDER_ID                13506 non-null  float64       
 2   SHOP_ID                 13506 non-null  int64         
 3   ADDRESS_TO_COUNTRY      13505 non-null  object        
 4   ADDRESS_TO_REGION       12930 non-null  object        
 5   ORDER_DT                13506 non-null  datetime64[ns]
 6   FULFILLED_DT            12637 non-null  datetime64[ns]
 7   REPRINT_FLAG            103 non-null    float64       
 8   SALES_CHANNEL_TYPE_ID   13506 non-null  int64         
 9   TOTAL_COST              13506 non-null  int64         
 10  TOTAL_SHIPPING          13506 non-null  int64         
 11  MERCHANT_REGISTERED_DT  13506 non-null  datetime64[ns]
 12  SUB_IS_ACTIVE_FLAG      13506 non-null  bool  

In [46]:
# Read the printify products excel file into pandas dataframe
new_line_items = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Line items')
new_line_items.head(20)

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
5,1.750016e+06,30,Generic brand,Mug,on-hold,1
6,4.668482e+05,10,Generic brand,Bags,shipment_delivered,1
7,1.569960e+06,45,Gildan,Sweatshirt,fulfilled,1
8,2.004430e+06,29,Gildan,Sweatshirt,shipment_delivered,1
9,2.255664e+05,25,Bella+Canvas,T-Shirt,shipment_delivered,1


In [47]:
new_line_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57583 entries, 0 to 57582
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ORDER_ID           57583 non-null  float64
 1   PRINT_PROVIDER_ID  57583 non-null  int64  
 2   PRODUCT_BRAND      57583 non-null  object 
 3   PRODUCT_TYPE       57583 non-null  object 
 4   ITEM_STATUS        57583 non-null  object 
 5   QUANTITY           57583 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 2.6+ MB


# Clean The data

## line_items cleaning

In [48]:
# Remove duplicates 
line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS'])

# DOn't remove orders with similar characteristics except quantity
#line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY'])
line_items.reset_index(drop=True, inplace=True)
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
15616,1.735807e+06,2,Generic brand,Canvas,shipment_delivered,1
15617,1.732949e+06,26,Gildan,T-Shirt,fulfilled,1
15618,1.449381e+06,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
15619,1.735807e+06,48,Generic brand,Mug,shipment_delivered,1


## Orders cleaning

In [49]:
# First rows encounter with duplicated order_id 
duplicate_orders1 = orders[orders['ORDER_ID'].duplicated(keep='last')]
duplicate_orders1

# Second rows encounter with duplicated order_id 
duplicate_orders2 = orders[orders['ORDER_ID'].duplicated(keep='first')]
duplicate_orders2

# Combine the duplicated rows into one dataframe for closer comparison
duplicates = pd.concat([duplicate_orders1, duplicate_orders2]).sort_values(by=['MERCHANT_ID'])
duplicates

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
537,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-24 17:34:42
538,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-28 16:51:06
5436,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,USPS,2020-10-16 19:44:40
5437,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,UPS_MAIL_INNOVATIONS,2020-10-19 19:21:56
13,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,USPS,2020-10-09 21:38:49
14,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,NaN,NaT
7502,7016547,1.555863e+06,1555863,US,TX,2020-10-29 22:10:53,2020-11-02 18:07:59,NaN,1,167,350,2020-02-17 23:36:26,True,business_account_3,USPS,2020-11-05 20:05:27
5285,7016547,1.555863e+06,1555863,US,LA,2020-10-11 01:33:06,2020-10-14 21:42:00,NaN,1,103,350,2020-02-17 23:36:26,True,business_account_3,NaN,NaT
6069,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-09 22:02:28
6070,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-13 20:28:54


In [50]:
orders.drop_duplicates(subset=['MERCHANT_ID', 'ORDER_ID'], 
                               inplace=True, keep='last')

In [51]:
# Not complete in removing the errors
orders.drop_duplicates(subset=['MERCHANT_ID', 'ORDER_ID', 
                               'SHOP_ID', 'ADDRESS_TO_COUNTRY', 
                               'ADDRESS_TO_REGION','ORDER_DT',
                               'FULFILLED_DT','REPRINT_FLAG',
                               'SALES_CHANNEL_TYPE_ID',
                               'TOTAL_COST','TOTAL_SHIPPING',
                               'MERCHANT_REGISTERED_DT',
                               'SUB_IS_ACTIVE_FLAG','SUB_PLAN', 
                               'SHIPMENT_CARRIER'], inplace=True,
                                  keep='last')
orders.reset_index(drop=True, inplace=True)

In [52]:
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


## Prepare for Joining

In [53]:
pd.options.mode.chained_assignment = None  # default='warn'
for i in range(len(line_items['ORDER_ID'])):
    
    # Split at the dot and get shop id (right) and order id (left)
    order = str(line_items.loc[i]['ORDER_ID']).split(".")
    
    # At same location insert the order id
    line_items.at[i, 'ORDER_ID'] = int(order[-1])
    
    # Add another column called shop id and add the right part
    # using this (shop_id) is the only way to join the data right
    line_items.at[i,'SHOP_ID'] = int(order[0])
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID
0,15208.0,30,Generic brand,Mug,on-hold,3,1750015.0
1,12357.0,30,Generic brand,Mug,on-hold,2,1750015.0
2,10081.0,16,Gildan,Kids clothes,shipment_delivered,1,1225155.0
3,707.0,39,Bella+Canvas,T-Shirt,shipment_delivered,1,1828890.0
4,55.0,10,Generic brand,Accessories,shipment_delivered,1,1847847.0
...,...,...,...,...,...,...,...
15616,1367.0,2,Generic brand,Canvas,shipment_delivered,1,1735807.0
15617,7852.0,26,Gildan,T-Shirt,fulfilled,1,1732948.0
15618,11854.0,29,Gildan,Sweatshirt,shipment_out_for_delivery,1,1449381.0
15619,2759.0,48,Generic brand,Mug,shipment_delivered,1,1735807.0


In [54]:
# Iteratively, mutate the order id to remove the shop id in the number
# The order id was concatenated with shop id using a dot making seem like
# a floating data type
for i in range(len(orders['ORDER_ID'])):
    
    # We convert the order id to string, split into 2 from where the dot is
    # The right from the dot section is shop id and left is order id
    order = str(orders.loc[i]['ORDER_ID']).split(".")
    
    # At same location, put the order id only, we already have the shop id on the dataset
    orders.at[i, 'ORDER_ID'] = int(order[-1])
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,4.0,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


# Join the Data

In [55]:
# Join the two dataframes to form one where each merchant is 
# Matched with all their orders and shops
printify_df = pd.merge(orders, line_items, how="inner", on=['ORDER_ID','SHOP_ID'])

In [56]:
printify_df

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,...,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,1,Generic brand,Mug,shipment_delivered,1
1,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,16,Generic brand,Mug,shipment_delivered,1
2,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,...,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56,39,Gildan,Sweatshirt,shipment_delivered,1
3,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,...,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54,25,Delta,T-Shirt,shipment_delivered,1
4,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,...,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37,16,Generic brand,Mug,shipment_delivered,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15616,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,...,2020-10-29 16:51:47,False,NaN,NaN,NaT,10,Generic brand,Accessories,canceled,10
15617,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,...,2020-09-30 23:11:41,True,business_account_3,NaN,NaT,16,Sport-Tek,Hats,on-hold,1
15618,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,...,2020-05-20 01:38:41,False,NaN,NaN,NaT,3,Gildan,Hoodie,on-hold,1
15619,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,...,2020-11-05 20:02:09,False,NaN,NaN,NaT,3,Gildan,Hoodie,canceled,1


In [57]:
printify_df['SHIPMENT_CARRIER'].unique()

array(['UPS', 'USPS', nan, 'UPS_MAIL_INNOVATIONS', 'DHL', 'GLOBEGISTICS',
       'ROYAL_MAIL', 'CANADA_POST', 'unknown', 'ASENDIA', 'RAYAL_MAIL',
       'DPD', 'DHL_GERMANY', 'FEDEX', 'ups_', 'DHL_EXPRESS',
       'OSM_WORLDWIDE'], dtype=object)

In [58]:
max_day = printify_df['SHIPMENT_DELIVERD_DT'].max() + pd.DateOffset(hours=100)
print(printify_df['SHIPMENT_DELIVERD_DT'].max())
print(max_day)

2021-01-26 20:13:23
2021-01-31 00:13:23


In [59]:
# In the case a shipment is not delivered, deliverly is assumed to take 
# place in the next 4 days hence the max day value for null deliver date
max_day = printify_df['SHIPMENT_DELIVERD_DT'].max() + pd.DateOffset(hours=100)

printify_df_temp1 = printify_df[['SHIPMENT_CARRIER', 'SHIPMENT_DELIVERD_DT','FULFILLED_DT']]
printify_df_temp1['Time_delay_hours'] = (printify_df['SHIPMENT_DELIVERD_DT'].fillna(max_day) - printify_df['FULFILLED_DT']).astype('timedelta64[h]')

shippment_delay = printify_df_temp1[['SHIPMENT_CARRIER', 'Time_delay_hours']].groupby(['SHIPMENT_CARRIER']).mean()
shippment_delay

,Time_delay_hours
SHIPMENT_CARRIER,
ASENDIA,1551.500000
CANADA_POST,221.128319
DHL,102.185259
DHL_EXPRESS,62.000000
DHL_GERMANY,1712.506849
DPD,103.539474
FEDEX,578.000000
GLOBEGISTICS,1437.248555
OSM_WORLDWIDE,141.000000


In [60]:
# Why on earth would some carriers have 0.0 delays. Impossible
# Solution is to fill in the null values (shipment not delivered) 
# with the last / max day of entry to be fair to those that delivered
printify_df[(printify_df['SHIPMENT_CARRIER'] == 'RAYAL_MAIL') | 
            (printify_df['SHIPMENT_CARRIER'] == 'ROYAL_MAIL') |
            (printify_df['SHIPMENT_CARRIER'] == 'DHL_GERMANY')]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,...,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
35,6948170,7466.0,1468748,DE,RHEINLAND-PFALZ,2020-11-07 04:24:34,2020-11-09 14:00:59,NaN,6,713,...,2019-12-26 07:22:20,True,business_account_3,ROYAL_MAIL,NaT,23,Premium Cases,Phone Case,fulfilled,1
38,7453498,195.0,2094464,GB,ENG,2020-11-05 15:05:47,2020-11-11 21:56:21,NaN,9,944,...,2020-09-27 14:57:40,True,business_account_3,ROYAL_MAIL,NaT,6,Gildan,T-Shirt,fulfilled,1
119,6860545,279.0,1366702,GB,SHROPSHIRE,2020-12-04 16:08:49,2020-12-11 22:08:00,NaN,4,1115,...,2019-10-11 15:57:25,True,business_account_3,ROYAL_MAIL,NaT,6,Gildan,T-Shirt,fulfilled,1
227,6584043,2645.0,1241313,GB,MID GLAMORGAN,2020-12-20 21:52:49,2021-01-05 21:24:25,NaN,4,1731,...,2019-05-12 21:34:43,True,business_account_3,ROYAL_MAIL,NaT,6,AWDIS,Sweatshirt,fulfilled,1
329,6652929,3554.0,1122627,GB,NOFOLK,2020-10-22 02:32:40,2020-10-26 22:22:39,NaN,4,1148,...,2019-06-06 23:10:51,True,business_account_3,RAYAL_MAIL,NaT,6,Bella+Canvas,T-Shirt,fulfilled,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15132,7389523,6.0,2022186,DE,NaN,2020-12-01 15:44:20,2020-12-16 14:10:10,NaN,1,3750,...,2020-09-11 19:38:49,False,NaN,DHL_GERMANY,NaT,26,B&C,Sweatshirt,fulfilled,1
15231,7291940,2839.0,1906136,DE,NaN,2020-12-16 21:17:03,2020-12-21 02:43:59,NaN,1,1872,...,2020-08-06 23:05:08,True,business_account_3,DHL_GERMANY,NaT,26,AWDIS,Hoodie,fulfilled,1
15413,6446058,304.0,897609,GB,NaN,2020-12-25 10:23:49,2021-01-04 22:38:38,NaN,4,1354,...,2019-03-25 22:15:11,False,NaN,ROYAL_MAIL,NaT,6,Bella+Canvas,T-Shirt,fulfilled,1
15452,6999280,16136.0,1533962,GB,ARGYLL,2020-11-18 09:52:06,2020-11-23 21:55:31,NaN,2,1098,...,2020-02-05 03:12:59,True,business_account_3,ROYAL_MAIL,NaT,6,Gildan,T-Shirt,fulfilled,1


In [61]:
# Capture the needed columns 
printify_df_temp = printify_df[['SHIPMENT_CARRIER','ITEM_STATUS']]

# If a shipment status faals in this category, it is considered successful
# and classified as delivered
carrier_delivered = printify_df_temp[(printify_df['ITEM_STATUS'] == 'shipment_delivered') | 
                                     (printify_df['ITEM_STATUS'] == 'fulfilled') |
                                     (printify_df['ITEM_STATUS'] == 'shipment_in_transit') | 
                                     (printify_df['ITEM_STATUS'] == 'shipment_out_for_delivery') | 
                                     (printify_df['ITEM_STATUS'] == 'shipment_available_for_pickup') | 
                                     (printify_df['ITEM_STATUS'] == 'in-production') | 
                                     (printify_df['ITEM_STATUS'] == 'on-hold') |
                                     (printify_df['ITEM_STATUS'] == 'shipment_pre_transit') |
                                     (printify_df['ITEM_STATUS'] == None)].groupby(['SHIPMENT_CARRIER']).count()

# This are considered canceled on the fault of a shipping carrier
# no 'canceled' attribute is counted since that is not a carriers fault.
carrier_delivered['Canceled'] = (printify_df_temp[
                                                  (printify_df['ITEM_STATUS'] == 'shipment_error') |
                                                  (printify_df['ITEM_STATUS'] == 'shipment_cancelled') |
                                                  (printify_df['ITEM_STATUS'] == 'shipment_return_to_sender') | 
                                                  (printify_df['ITEM_STATUS'] == 'shipment_failure')].groupby(['SHIPMENT_CARRIER']).count()).fillna(0)

carrier_delivered.rename(columns={'ITEM_STATUS':'Derivered'}, inplace=True)
carrier_delivered.fillna(0)

,Derivered,Canceled
SHIPMENT_CARRIER,,
ASENDIA,179,0.0
CANADA_POST,223,3.0
DHL,499,1.0
DHL_EXPRESS,9,0.0
DHL_GERMANY,73,0.0
DPD,76,0.0
FEDEX,2,0.0
GLOBEGISTICS,346,0.0
OSM_WORLDWIDE,1,0.0


In [62]:
# Interesting case from above where no shipment is delivered yet
# This is not used later since it is captured int the canceled column above
printify_df_temp = printify_df[['SHIPMENT_CARRIER', 'QUANTITY','ITEM_STATUS']]

# This are considered canceled on the fault of a shipping carrier
# no 'canceled' attribute is counted since that is not a carriers fault.
shipment_failure = printify_df_temp[(printify_df['ITEM_STATUS'] == 'shipment_error') |
                                                  (printify_df['ITEM_STATUS'] == 'shipment_cancelled') |
                                                  (printify_df['ITEM_STATUS'] == 'shipment_return_to_sender') | 
                                                  (printify_df['ITEM_STATUS'] == 'shipment_failure')].groupby(['SHIPMENT_CARRIER']).count()

shipment_failure.rename(columns={'ITEM_STATUS':'Shipment_failed'}, inplace=True)
shipment_failure

,QUANTITY,Shipment_failed
SHIPMENT_CARRIER,,
CANADA_POST,3,3
DHL,1,1
UPS,1,1
USPS,30,30


In [63]:
# Capture the data needed
printify_df_temp1 = printify_df[['SHIPMENT_CARRIER', 'TOTAL_COST','TOTAL_SHIPPING','QUANTITY']]

# create a column whose value is the total shipping cost divided by the number of orders delivered 
printify_df_temp1['Cost_per_product_X1000'] = ((printify_df['TOTAL_SHIPPING'] / printify_df['QUANTITY'])) / 1000

# Get the sum of all the columns grouped by the carrier provider
shippment_cost = printify_df_temp1[['SHIPMENT_CARRIER', 'QUANTITY', 'TOTAL_COST', 'TOTAL_SHIPPING','Cost_per_product_X1000']].groupby(['SHIPMENT_CARRIER']).sum()
shippment_cost

,QUANTITY,TOTAL_COST,TOTAL_SHIPPING,Cost_per_product_X1000
SHIPMENT_CARRIER,,,,
ASENDIA,181,307732,262208,259.973000
CANADA_POST,258,518687,227617,215.754163
DHL,646,1446024,760492,675.588361
DHL_EXPRESS,10,43230,33380,32.680000
DHL_GERMANY,74,139338,37459,37.219000
DPD,79,86126,49360,47.605000
FEDEX,2,3224,5024,5.024000
GLOBEGISTICS,356,643736,570460,558.740000
OSM_WORLDWIDE,1,739,400,0.400000


In [64]:
# Get the mode of country column for data grouped for each carrier
carrier_poplular_country = printify_df[['SHIPMENT_CARRIER', 'ADDRESS_TO_COUNTRY']].groupby(['SHIPMENT_CARRIER'], sort=True).agg(pd.Series.mode)

# Rename columns
carrier_poplular_country.rename(columns={'ADDRESS_TO_COUNTRY':'Popular_country'}, inplace=True)
carrier_poplular_country

,Popular_country
SHIPMENT_CARRIER,
ASENDIA,CA
CANADA_POST,CA
DHL,US
DHL_EXPRESS,US
DHL_GERMANY,DE
DPD,DE
FEDEX,US
GLOBEGISTICS,CA
OSM_WORLDWIDE,US


In [65]:
# Get the mode of product column for data grouped for each carrier
carrier_poplular_product = printify_df[['SHIPMENT_CARRIER', 'PRODUCT_TYPE']].groupby(['SHIPMENT_CARRIER'], sort=True).agg(pd.Series.mode)

# Rename columns
carrier_poplular_product.rename(columns={'PRODUCT_TYPE':'Popular_product'}, inplace=True)
carrier_poplular_product

,Popular_product
SHIPMENT_CARRIER,
ASENDIA,T-Shirt
CANADA_POST,T-Shirt
DHL,Mug
DHL_EXPRESS,Poster
DHL_GERMANY,T-Shirt
DPD,Mug
FEDEX,"[All Over Prints, Home Decor]"
GLOBEGISTICS,T-Shirt
OSM_WORLDWIDE,T-Shirt


In [66]:
# Get the mode of print providers column for data grouped for each carrier
carrier_poplular_printer = printify_df[['SHIPMENT_CARRIER', 'PRINT_PROVIDER_ID']].groupby(['SHIPMENT_CARRIER'], sort=True).agg(pd.Series.mode)

# Rename columns
carrier_poplular_printer.rename(columns={'PRINT_PROVIDER_ID':'Popular_printer'}, inplace=True)
carrier_poplular_printer

,Popular_printer
SHIPMENT_CARRIER,
ASENDIA,3
CANADA_POST,41
DHL,1
DHL_EXPRESS,36
DHL_GERMANY,26
DPD,30
FEDEX,1
GLOBEGISTICS,29
OSM_WORLDWIDE,3


In [67]:
# Get the mode of merchants column for data grouped for each carrier
carrier_poplular_merchant = printify_df[['SHIPMENT_CARRIER', 'MERCHANT_ID']].groupby(['SHIPMENT_CARRIER'], sort=True).agg(pd.Series.mode)

# Rename columns
carrier_poplular_merchant.rename(columns={'MERCHANT_ID':'Popular_merchant'}, inplace=True)
carrier_poplular_merchant

,Popular_merchant
SHIPMENT_CARRIER,
ASENDIA,7160379
CANADA_POST,7143681
DHL,5755049
DHL_EXPRESS,5755049
DHL_GERMANY,7156242
DPD,7019548
FEDEX,"[6062712, 6612042]"
GLOBEGISTICS,6799169
OSM_WORLDWIDE,6172284


In [68]:
# Join the popular traits before joining with teh rest
carrier_poplulars1 =  pd.merge(carrier_poplular_country, carrier_poplular_product, on=['SHIPMENT_CARRIER'])
carrier_poplulars2 =  pd.merge(carrier_poplulars1, carrier_poplular_printer, on=['SHIPMENT_CARRIER'])
carrier_poplulars =  pd.merge(carrier_poplulars2, carrier_poplular_merchant, on=['SHIPMENT_CARRIER'])
carrier_poplulars

,Popular_country,Popular_product,Popular_printer,Popular_merchant
SHIPMENT_CARRIER,,,,
ASENDIA,CA,T-Shirt,3,7160379
CANADA_POST,CA,T-Shirt,41,7143681
DHL,US,Mug,1,5755049
DHL_EXPRESS,US,Poster,36,5755049
DHL_GERMANY,DE,T-Shirt,26,7156242
DPD,DE,Mug,30,7019548
FEDEX,US,"[All Over Prints, Home Decor]",1,"[6062712, 6612042]"
GLOBEGISTICS,CA,T-Shirt,29,6799169
OSM_WORLDWIDE,US,T-Shirt,3,6172284


# Shipping Carriers Performance Metrics

In [69]:
# Merge the individual printer performance into one df 
# Merge was preffered over concat due to precision and ability
# to specify the collumn to merge on like in SQL
# Thus, not very clean code here, but very optimal as others would
# need to sort first

# Merge shipment delay with orders delivered df
carrier_perf1 = pd.merge(shippment_delay, carrier_delivered, on=['SHIPMENT_CARRIER'])

# join previous with fiscal matters df
carrier_perf2 = pd.merge(carrier_perf1, shippment_cost, on=['SHIPMENT_CARRIER'])

# join the popular traits for each carrier
carrier_perf = pd.merge(carrier_perf2, carrier_poplulars, on=['SHIPMENT_CARRIER'])

carrier_perf

,Time_delay_hours,Derivered,Canceled,QUANTITY,TOTAL_COST,TOTAL_SHIPPING,Cost_per_product_X1000,Popular_country,Popular_product,Popular_printer,Popular_merchant
SHIPMENT_CARRIER,,,,,,,,,,,
ASENDIA,1551.500000,179,NaN,181,307732,262208,259.973000,CA,T-Shirt,3,7160379
CANADA_POST,221.128319,223,3.0,258,518687,227617,215.754163,CA,T-Shirt,41,7143681
DHL,102.185259,499,1.0,646,1446024,760492,675.588361,US,Mug,1,5755049
DHL_EXPRESS,62.000000,9,NaN,10,43230,33380,32.680000,US,Poster,36,5755049
DHL_GERMANY,1712.506849,73,NaN,74,139338,37459,37.219000,DE,T-Shirt,26,7156242
DPD,103.539474,76,NaN,79,86126,49360,47.605000,DE,Mug,30,7019548
FEDEX,578.000000,2,NaN,2,3224,5024,5.024000,US,"[All Over Prints, Home Decor]",1,"[6062712, 6612042]"
GLOBEGISTICS,1437.248555,346,NaN,356,643736,570460,558.740000,CA,T-Shirt,29,6799169
OSM_WORLDWIDE,141.000000,1,NaN,1,739,400,0.400000,US,T-Shirt,3,6172284


In [70]:
carrier_perf.describe()

,Time_delay_hours,Derivered,Canceled,QUANTITY,TOTAL_COST,TOTAL_SHIPPING,Cost_per_product_X1000
count,16.000000,16.000000,4.000000,16.000000,1.600000e+01,1.600000e+01,16.000000
mean,714.345932,856.125000,8.750000,922.937500,1.757028e+06,8.601811e+05,826.148079
std,721.730331,2727.951243,14.198005,2916.710738,5.458547e+06,2.519396e+06,2426.622563
min,62.000000,1.000000,1.000000,1.000000,7.390000e+02,4.000000e+02,0.400000
25%,103.200920,9.000000,1.000000,11.500000,3.566325e+04,2.795150e+04,27.160250
50%,255.688668,127.000000,2.000000,129.000000,1.982980e+05,1.131375e+05,110.969000
75%,1524.615385,253.750000,9.750000,282.500000,5.499492e+05,3.994450e+05,385.741550
max,1772.142857,11056.000000,30.000000,11824.000000,2.212304e+07,1.024193e+07,9864.838624


# Shipping carriers Success Metric
    - bundling_ratio is the ability of a carrier to bundle more than one shipment orders during delivery. On average it is given by the total sum of all orders delivered and the number of recorded deliveries. If for every order, there was a trip to deliver, this ratio will be 1 hence no effect on the success metrics of the carrier. 
The success metric is given by successful orders delivered i.e. (derivered - canceled) orders multiplied by the bundling ratio. All that will be divided by the sum of shipment cost per product and delay time since these are inversly proportional to success of a carrier.

In [71]:
bundling_ratio = carrier_perf['QUANTITY'] / carrier_perf['Derivered'].fillna(0)
carrier_perf['Success_factor'] = ((((carrier_perf['Derivered'].fillna(0) - carrier_perf['Canceled'].fillna(0))*2)* bundling_ratio) /
                 np.log10(carrier_perf['Cost_per_product_X1000'] + carrier_perf['Time_delay_hours'])) 
carrier_perf.sort_values(['Success_factor'], ascending=False, inplace=True)
carrier_perf

,Time_delay_hours,Derivered,Canceled,QUANTITY,TOTAL_COST,TOTAL_SHIPPING,Cost_per_product_X1000,Popular_country,Popular_product,Popular_printer,Popular_merchant,Success_factor
SHIPMENT_CARRIER,,,,,,,,,,,,
USPS,165.013997,11056,30.0,11824,22123039,10241931,9864.838624,US,T-Shirt,29,5755049,5894.050494
UPS_MAIL_INNOVATIONS,290.249017,764,NaN,830,1916194,989639,955.385710,US,Poster,2,6852170,536.281251
DHL,102.185259,499,1.0,646,1446024,760492,675.588361,US,Mug,1,5755049,446.031233
GLOBEGISTICS,1437.248555,346,NaN,356,643736,570460,558.740000,CA,T-Shirt,29,6799169,215.747243
CANADA_POST,221.128319,223,3.0,258,518687,227617,215.754163,CA,T-Shirt,41,7143681,192.798483
UPS,81.632124,200,1.0,229,505548,342440,328.075400,US,Canvas,2,7158456,174.436190
ASENDIA,1551.500000,179,NaN,181,307732,262208,259.973000,CA,T-Shirt,3,7160379,111.110024
ROYAL_MAIL,1515.653846,133,NaN,134,257258,118928,117.966000,GB,T-Shirt,6,"[6999280, 7453498]",83.407222
unknown,1606.512397,121,NaN,124,99857,107347,103.972000,GB,Mug,9,1819516,76.706113


## How other metrics correlate with sucess factor

In [72]:
carrier_perf['Success_factor'].corr(carrier_perf['QUANTITY'])

0.9996082997851286

In [73]:
# using plotly express to plot an interactive scatter plot
fig = px.scatter(carrier_perf, x="Derivered", y="Success_factor", 
                 trendline="ols", size="TOTAL_COST",
                 title='Carrier success vs total number of delivered orders')
fig.show()

In [74]:
carrier_perf['Success_factor'].corr(carrier_perf['Time_delay_hours'])

-0.22748017642042379

In [75]:
# using plotly express to plot an interactive scatter plot
fig = px.scatter(carrier_perf, x="Time_delay_hours", y="Success_factor", 
                 trendline="ols", size="TOTAL_COST", 
                 title='Carrier sucess vs time delay before deliverly')
fig.show()

# What are the top two shipping carriers? 

### They are:
- UPS_MAIL_INNOVATIONS
- USPS

# Why should or shouldn’t we try to use those two for all orders?


In [76]:
tr = carrier_perf
val = carrier_perf['QUANTITY']

fig = px.pie(tr, values=val, names=tr.index, title='Shipping Carrier quantity delivered comparison', hole=0.3)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

In [77]:
print(f"Top two orders delivered is {(carrier_perf.iloc[:2,1].sum() / carrier_perf['Derivered'].sum()) * 100}% of the total orders delivered")
print(f"Top two total revenue is {(carrier_perf.iloc[:2,4].sum() / carrier_perf['TOTAL_COST'].sum()) * 100}% of the total quantity")

Top two orders delivered is 86.28996933858957% of the total orders delivered
Top two total revenue is 85.51098230460232% of the total quantity


From the above finding, using the two most successful carriers is the most optimal strategy. 
# However ...
from the following crucial info below


In [78]:
# Best two
carrier_perf.iloc[:2]

,Time_delay_hours,Derivered,Canceled,QUANTITY,TOTAL_COST,TOTAL_SHIPPING,Cost_per_product_X1000,Popular_country,Popular_product,Popular_printer,Popular_merchant,Success_factor
SHIPMENT_CARRIER,,,,,,,,,,,,
USPS,165.013997,11056,30.0,11824,22123039,10241931,9864.838624,US,T-Shirt,29,5755049,5894.050494
UPS_MAIL_INNOVATIONS,290.249017,764,NaN,830,1916194,989639,955.385710,US,Poster,2,6852170,536.281251


### The top two carriers mostly:
- Deliver to the United States Printify market
- Deliver T-shirts and posters
- Delver for print providers 29 and 2
- Deliver orders for merchant 5755049 and 6852170

# USPS and UPS_MAIL_INNOVATIONS might be the best but other carriers also play a crucial role in the business model of printify.
# Many carrier services increase the geological penetration of printify, making more products available for the customers, and making the print providers more successful which is an advantage to Printify.
# Its not a coincidence that US is the biggest market for Printify and the two most successful carriers mostly offer their services there.
# A balance should be struck to make all the factors operate at an optimum level i.e. Increase reach to many countries, delivering many orders, delivering many different products in the catalogue, and serving many merchants / print providers.